### Notebook to run *cis*-eQTL analysis using [tensorQTL](https://github.com/broadinstitute/tensorqtl)

[Taylor-Weiner, Aguet, et al., Genome Biol. 20:228, 2019.](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-019-1836-7)


In [ ]:
!date

In [ ]:
!source myconda

#### import libraries and set notebook variables

In [ ]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
import torch
import tensorqtl.tensorqtl as tensorqtl
from tensorqtl.tensorqtl import genotypeio, cis, trans
print('PyTorch {}'.format(torch.__version__))
print('Pandas {}'.format(pd.__version__))
from pgenlib import PgenReader
import pgenlib as pgen
import seaborn as sns
from kneed import KneeLocator 

import os
import statsmodels.stats.multitest as smm
import scikit_posthocs as sp
import statsmodels
import statsmodels.api as sm
import itertools
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
cohort = "nabec"
version = "Jun_2024"
target = "cpg_islands"
varianttype_caller = "SV_harmonized"

In [ ]:
# naming
modality = 'METH'
#cohort_build = f'{cohort}.{version}.{target}'
set_name = f'{cohort}_{version}_{target}_{varianttype_caller}'
#make sure if using both SV and SNV, use "SNV_SV_{caller}
cohort_version_target = f'{cohort}_{version}_{target}'


# directories
in_dir = f'/data/CARDPB/data/NABEC/projects/QTL_paper_2024/SV-mQTL'
geno_dir = f'{in_dir}/genotypes/{varianttype_caller}'
quants_dir = f'{in_dir}/expression'
info_dir = f'{in_dir}/sample_info'
tensorqtl_dir = f'/data/CARDPB/data/NABEC/projects/QTL_paper_2024/newSV-mQTL/tenosorqtl/{set_name}'
results_dir = f'/data/CARDPB/data/NABEC/projects/QTL_paper_2024/newSV-mQTL/results/{set_name}'


#for RNA (used SV PC1-5 and SNV PC 1-5 for SV and SNV)  'SNVPC1', 'SNVPC2', 'SNVPC3', 'SNVPC4', 'SNVPC5', 
#for RNA (used SV PC1-5 and SNV PC 1-5 for SV and SNV)  'SNVPC1', 'SNVPC2', 'SNVPC3', 'SNVPC4', 'SNVPC5', 
if varianttype_caller == 'SNV_illumina':
    covs_columns_to_use = ['female','PMI', 'Age', 'JHU', 'MIAMI', 'SH', 'UKY', 'UMARY']
    bfile_prefix_path = f'{geno_dir}/MERGED_MAF_GENO005_HWE_0001_ONT_plink19_Jul2024'
    gPCA_path = f'{geno_dir}/MERGED_MAF_GENO005_HWE_0001_ONT_plink19_Jul2024_prun_pca20.txt'
    eigenvalues = pd.read_csv('/data/CARDPB/data/NABEC/projects/QTL_paper_2024/SV-eQTL/genotypes/SNV_illumina/MERGED_MAF_GENO005_ONT_plink19_pca20.eigenval',header=None)
elif varianttype_caller ==  'SV_harmonized':
    covs_columns_to_use = ['female','PMI', 'Age', 'JHU', 'MIAMI', 'SH', 'UKY', 'UMARY']
    bfile_prefix_path = f'{geno_dir}/nabec_GENO_MAF_005_HWE_0001_updateid'
    gPCA_path = f'{geno_dir}/nabec_GENO_MAF_005_HWE_0001_updateid_prun_pca20.txt'
    eigenvalues = pd.read_csv('/data/CARDPB/data/NABEC/projects/QTL_paper_2024/SV-eQTL/genotypes/SV_harmonized/nabec_GENO_MAF_005_HWE_0001_updateid_prun_pca20.eigenval',header=None)

elif varianttype_caller ==  'SV_harmonized_SNV':
    covs_columns_to_use = ['female','PMI', 'Age', 'JHU', 'MIAMI', 'SH', 'UKY', 'UMARY']
    bfile_prefix_path = f'{geno_dir}/harmonized_SV_SNV_MAF_GENO_005_HWE_0001'
    gPCA_path = f'{geno_dir}/harmonized_SV_SNV_MAF_GENO_005_HWE_0001_prun_pca20.txt'
    eigenvalues = pd.read_csv('/data/CARDPB/data/NABEC/projects/QTL_paper_2024/SV-eQTL/genotypes/SV_harmonized_SNV/harmonized_SV_SNV_MAF_GENO_005_HWE_0001_prun_pca20.eigenval',header=None)

# input file
quants_bed_file = f'{quants_dir}/{cohort_version_target}.scaled.bed.gz'm
assay_covs_files = f'{info_dir}/nabec.aug2020.sample_info.txt'

# output files
used_samples_list_file = f'{info_dir}/{set_name}.samples'
cis_indep_file = f'{results_dir}/{set_name}.cis.indep.csv'
cis_map_file = f'{tensorqtl_dir}/{set_name}.cis.map.csv'
qtnormalized_expression_pc = f'{info_dir}/{cohort_version_target}.normPC.csv'

# constant values
alpha_value = 0.05
min_nominal_alpha = 1e-05
# tensorQTL defaults to 0
MIN_MAF = 0
DEBUG=False

In [ ]:
os.makedirs(tensorqtl_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

#### utility functions

In [ ]:
# compute B&H FDR for given p-values #pvalue multiple test
def compute_fdr(pvalues):
    bh_adj = smm.fdrcorrection(pvalues)
    return bh_adj[1]

#### load phenotypes and covariates (if needed)

In [ ]:
%%time

phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed(quants_bed_file)
print(f'phenotype_df {phenotype_df.shape}')
print(f'phenotype_pos_df {phenotype_pos_df.shape}')
# display(phenotype_df.head())
# display(phenotype_pos_df.head())

## load covariates and format

In [ ]:
covs_df = pd.read_csv(assay_covs_files, index_col=0)
print(f'covariates shape {covs_df.shape}')
#if DEBUG:
#    display(covs_df.head())

In [ ]:
covs_df

#### create a binarized covariate for sex

In [ ]:
covs_df['female'] = 0
covs_df.loc[covs_df.Sex == 'female', 'female'] = 1
display(covs_df.Sex.value_counts())
display(covs_df.female.value_counts())

In [ ]:
#get dummies to the Group
onehot_batch = pd.get_dummies(covs_df.Group, drop_first=True)
# should have the same index
print(f'indices are equal: {covs_df.index.equals(onehot_batch.index)}')
covs_df = pd.concat([covs_df, onehot_batch], axis=1)
print(f'new covariates shape: {covs_df.shape}')
#if DEBUG:
#    display(onehot_batch.sample(5))
#    display(covs_df.sample(5))

### merge genetic-PCA

In [ ]:
g_pca = pd.read_csv(f"{gPCA_path}", index_col=1, sep=' ').drop('0',axis=1)

In [ ]:
g_pca['SampleId'] = [i.replace('NABEC_','').replace('_FTX','') for i in g_pca.index.to_list()]

In [ ]:
covs_df = covs_df.merge(g_pca,on="SampleId")

In [ ]:
#plot PCA of geno
#N 20
pca_row = g_pca.drop('SampleId',axis=1)
plt.figure(figsize=(6, 6))
plt.scatter(pca_row.iloc[:, 0], pca_row.iloc[:, 1], alpha=0.8)
plt.grid()
plt.xlabel("PC1")
plt.ylabel("PC2")
x = pca_row.iloc[:, 0]
y = pca_row.iloc[:, 1]
annotations = pca_row.index
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i], y[i]))
plt.show()

#eigenvalues = pd.read_csv('/data/CARDPB/data/NABEC/projects/QTL_paper_2024/SV-eQTL/genotypes/SNV_illumina/MERGED_MAF_GENO005_ONT_plink19_pca20.eigenval',header=None)
print(eigenvalues)
import numpy as np
import matplotlib.pyplot as plt


# calculate cotriburtion rates
total_variance = np.sum(eigenvalues)
contribution_rates = (eigenvalues / total_variance).to_numpy().flatten()

# change to list
n_components = np.arange(1, len(contribution_rates) + 1)

# make instance by KneeLocator
knee_locator = KneeLocator(n_components, contribution_rates, curve='convex', direction='decreasing')

knee_locator.plot_knee()

# output the knee point
print(f'Optimal number of components: {knee_locator.knee}')

plt.show()

GPCA_NUM = knee_locator.knee

### merge expression PCA


In [ ]:
phenotype_df = phenotype_df.dropna()

In [74]:
temp_EXP = phenotype_df.T

In [ ]:
#plot PCA of meth
#N 20
pca = PCA(n_components=20)
pca.fit(temp_EXP)

pca_row = pca.transform(temp_EXP)

plt.figure(figsize=(6, 6))
plt.scatter(pca_row[:, 0], pca_row[:, 1], alpha=0.8)
plt.grid()
plt.xlabel("PC1")
plt.ylabel("PC2")
x = pca_row[:, 0]
y = pca_row[:, 1]
annotations = temp_EXP.index
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i], y[i]))
plt.show()

eigenvalues = pd.DataFrame(pca.explained_variance_ratio_)

import numpy as np
import matplotlib.pyplot as plt


# calc contribution
total_variance = np.sum(eigenvalues)
contribution_rates = (eigenvalues / total_variance).to_numpy().flatten()

# change x-axis to list
n_components = np.arange(1, len(contribution_rates) + 1)

# make instance by KneeLocator
knee_locator = KneeLocator(n_components, contribution_rates, curve='convex', direction='decreasing')

knee_locator.plot_knee()

# output the knee point
print(f'Optimal number of components: {knee_locator.knee}')

plt.show()

PHENOPCA_NUM = knee_locator.knee

PHENOPCS = [f'{modality}_PC_'+str(i) for i in range(1,PHENOPCA_NUM+1)] 
covs_columns_to_use += PHENOPCS

In [ ]:
GENOPCS = [f'GENPC'+str(i) for i in range(1,GPCA_NUM+1)] 
covs_columns_to_use += GENOPCS

columns_PC = []
for i in range(1,21):
    columns_PC.append(f"{modality}_PC_"+ str(i))

In [ ]:
df_EXP_PC = pd.DataFrame(pca_row,index=temp_EXP.index, columns=columns_PC)

In [ ]:
from sklearn import preprocessing
scaledX = preprocessing.quantile_transform(df_EXP_PC, axis=0, copy=True,
                                           output_distribution='normal')
qtnorm_EXP_PC = pd.DataFrame(data=scaledX, columns=df_EXP_PC.columns,
                               index=df_EXP_PC.index)

In [ ]:
qtnorm_EXP_PC

In [ ]:
qtnorm_EXP_PC.to_csv(qtnormalized_expression_pc)

In [ ]:
covs_df = covs_df.merge(qtnorm_EXP_PC,left_on="SampleId", right_index=True)

#### load plink pfiles

In [ ]:
columns_PC = []
for i in range(1,21):
    columns_PC.append(f"{modality}_PCA"+ str(i))

In [ ]:
df_EXP_PC = pd.DataFrame(pca_row,index=temp_EXP.index, columns=columns_PC)

In [ ]:
%%time

# pr = genotypeio.PlinkReader(bfile_prefix_path, select_samples=phenotype_df.columns)
pr = genotypeio.PlinkReader(bfile_prefix_path)
genotype_df = pr.load_genotypes()
variant_df = pr.bim.set_index('snp')[['chrom', 'pos']]

In [ ]:
variant_df.shape

In [ ]:
print(genotype_df.shape)
# display(genotype_df.head())
print(variant_df.shape)
# display(variant_df.head())

In [ ]:
# tensorQTL says wants plink bfiles, but wants bim chrs to include 'chr'
variant_df['chrom'] = 'chr' + variant_df['chrom']
print(variant_df.shape)
display(variant_df.head())

#### make sure the pheno and genos have same samples

In [ ]:
phenotype_df

In [ ]:
assay_intersect_samples = set(genotype_df.columns) & set(phenotype_df.columns) 
print(f'intersect {len(assay_intersect_samples)}')
extra_geno_samples = set(genotype_df.columns) - set(phenotype_df.columns)
print(f'number of genotypes samples not in expression {len(extra_geno_samples)}')
extra_expr_samples = set(phenotype_df.columns) - set(genotype_df.columns)
print(f'number of expression samples not in genotypes {len(extra_expr_samples)}')

# save the used sample list
pd.DataFrame(data=assay_intersect_samples).to_csv(used_samples_list_file, 
                                                  index=False, header=False)

#### drop the non-matched samples

In [ ]:
genotype_df.drop(columns=extra_geno_samples, inplace=True)
phenotype_df.drop(columns=extra_expr_samples, inplace=True)

print(genotype_df.shape)
# display(genotype_df.head())
print(phenotype_df.shape)
# display(phenotype_df.head())

#### need to make sure phenos and genos have matched chromosomes; ie just autosomes

In [ ]:
# need to ditch any non-autosomal genes
assay_intersect_chroms = set(phenotype_pos_df['chr']) & set(variant_df['chrom']) 
print(f'intersect {len(assay_intersect_chroms)}')
extra_geno_chroms = set(variant_df['chrom']) - set(phenotype_pos_df['chr'])
print(f'number of genotypes chroms not in expression {len(extra_geno_chroms)}')
print(extra_geno_chroms)
extra_expr_chroms = set(phenotype_pos_df['chr']) - set(variant_df['chrom'])
print(f'number of expression chroms not in genotypes {len(extra_expr_chroms)}')
print(extra_expr_chroms)

In [ ]:
if len(extra_geno_chroms) > 0:
    variant_df = variant_df.loc[~variant_df['chrom'].isin(extra_geno_chroms)]
    # this will remove variants so need to remove them from genos df as well
    genotype_df = genotype_df.loc[genotype_df.index.isin(variant_df.index)]
if len(extra_expr_chroms) > 0:
    phenotype_pos_df = phenotype_pos_df.loc[~phenotype_pos_df['chr'].isin(extra_expr_chroms)]
    # this will remove genes so need to remove them from phenos df as well
    phenotype_df = phenotype_df.loc[phenotype_df.index.isin(phenotype_pos_df.index)]

print(genotype_df.shape)
# display(genotype_df.head())
print(variant_df.shape)
# display(variant_df.head())
print(phenotype_df.shape)
# display(phenotype_df.head())
print(phenotype_pos_df.shape)
# display(phenotype_pos_df.head())

### make sure covariates match geno and pheno samples

In [ ]:
# subest covs to just this 'day'; ie all differention days covs in file
# also since only interested in cell fractions as interaction terms, subset now
covs_df = covs_df.loc[covs_df.SampleId.isin(phenotype_df.columns)]
print(f'covs shape {covs_df.shape}')

cov_intersect_samples = set(phenotype_df.columns) & set(covs_df.SampleId) 
print(f'intersect {len(cov_intersect_samples)}')
extra_expr_samples = set(phenotype_df.columns) - set(covs_df.SampleId)
print(f'number of endogenous samples not in covariates {len(extra_expr_samples)}')
extra_cov_samples = set(covs_df.SampleId) - set(phenotype_df.columns)
print(f'number of covariate samples not in exogenous {len(extra_cov_samples)}')

#### subset covariate to just desired (ie cell fractions) and shape for use with tensorqtl

In [ ]:
covs_df.columns

In [ ]:
covs_to_use = covs_df[['SampleId'] + covs_columns_to_use]
covs_to_use.drop_duplicates(subset=['SampleId'], keep='first', inplace=True)
covs_to_use.set_index('SampleId', inplace=True)
# re-order columns to match phenotypes
covs_to_use = covs_to_use.transpose()
covs_to_use = covs_to_use[phenotype_df.columns]
# now transpose back
covs_to_use = covs_to_use.transpose()
print(covs_to_use.shape)
#if DEBUG:
#    display(covs_to_use.head())

#### in rare instances a single sample will be missing a covariate, mean fill for simplicity

In [ ]:
for covariate in covs_to_use.columns:
    mean_val = covs_to_use[covariate].mean()
    if covs_to_use[covariate].nunique() == 2:
        mean_val = int(mean_val)
    covs_to_use[covariate].fillna(mean_val, inplace=True)
print(covs_to_use.shape)
if DEBUG:
    display(covs_to_use.head())

In [ ]:
covs_to_use.drop(['JHU','MIAMI'],inplace=True, axis=1)

In [ ]:
for covariate in covs_to_use.columns:
    mean_val = covs_to_use[covariate].mean()
    print(covariate)
    print(mean_val)

#### *cis*-QTL: nominal p-values for all variant-phenotype pairs

In [ ]:
%%time
# map all cis-associations (results for each chromosome are written to file)
# all features
cis.map_nominal(genotype_df, variant_df,  phenotype_df, phenotype_pos_df, 
                covariates_df=covs_to_use, prefix=set_name, output_dir=tensorqtl_dir, 
                run_eigenmt=True, write_top=True, write_stats=True, verbose=False, maf_threshold=MIN_MAF)

#### *cis*-QTL: empirical p-values for phenotypes

In [ ]:
%%time
# all genes
cis_df = cis.map_cis(genotype_df, variant_df, phenotype_df, phenotype_pos_df,covs_to_use, 
                     warn_monomorphic=False, verbose=False, maf_threshold=MIN_MAF)

# don't have to replace the monorphic anymore tensorqtl added flag to silence
# note I commented out the following bit of code in tensorqtl/cis.py to reduce log spill
# logger.write('    * WARNING: excluding {} monomorphic variants'.format(mono_t.sum()))

# commented printing this exception in core.py to reduce non-log spill
# print('WARNING: scipy.optimize.newton failed to converge (running scipy.optimize.minimize)')

#### compute the FDR

In [ ]:
# add the corrected p-value, note just based on all chrom features pvalues    
# just using B&H FDR from statsmodel is approx equivalent to Storey qvalue, tested
cis_df['bh_fdr'] = compute_fdr(cis_df['pval_beta'].fillna(1))

# tensorQTL uses qvalue, but requires the R packages so use above BH FDR instead to approx
tensorqtl.calculate_qvalues(cis_df, qvalue_lambda=0.85)

In [ ]:
print(cis_df.shape)
display(cis_df.head())

In [135]:
print(cis_df.loc[cis_df['pval_nominal'] <= min_nominal_alpha].index.unique().shape)
print(cis_df.loc[cis_df['pval_perm'] <= alpha_value].index.unique().shape)
print(cis_df.loc[cis_df['pval_beta'] <= alpha_value].index.unique().shape)
print(cis_df.loc[cis_df['qval'] <= alpha_value].index.unique().shape)

(18,)
(34,)
(34,)
(3,)


#### save cis map

In [ ]:
%%time
cis_df.to_csv(cis_map_file)

#### map the loci independent signals

In [ ]:
# # use the B&H fdr instead of Storey qvalue
indep_df = cis.map_independent(genotype_df, variant_df, cis_df, phenotype_df, 
                                phenotype_pos_df, covs_to_use,
                                fdr_col='qval', verbose=False, maf_threshold=MIN_MAF)

In [ ]:
print(indep_df.shape)
display(indep_df.head())
print(indep_df['phenotype_id'].unique().shape)

In [ ]:
indep_df['rank'].value_counts()

#### save the loci independent signals

In [ ]:
indep_df.to_csv(cis_indep_file)

In [ ]:
indep_df.loc[indep_df['pval_nominal'] == indep_df['pval_nominal'].min()]

In [ ]:
!date